In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import csv
import os

import matplotlib
matplotlib.use('TkAgg')

In [259]:
# Mark Points
folder = "../calibration_data/thermal_dots_out1/"

files = os.listdir(folder)
points = dict()
for f in files:
	print(f)
	path = os.path.join(folder, f)
	plt.imshow(cv2.imread(path,  cv2.IMREAD_UNCHANGED))
	
	points[f] = plt.ginput(35, timeout=-1) # x, y
	print(len(points[f]))
	print(points[f], "\n")

Lepton_Capture.jpg
35
[(66.07342657342657, 90.44905094905094), (64.79470529470528, 77.82167832167832), (63.03646353646353, 64.87462537462537), (61.43806193806192, 51.92757242757243), (59.67982017982017, 39.140359640359634), (76.78271728271729, 91.24825174825175), (76.3031968031968, 78.62087912087912), (74.54495504495505, 64.23526473526474), (73.10639360639361, 50.96853146853147), (72.3071928071928, 37.7017982017982), (90.2092907092907, 91.24825174825175), (89.25024975024975, 77.34215784215783), (88.13136863136863, 64.87462537462537), (87.33216783216783, 49.68981018981019), (84.45504495504495, 33.705794205794206), (104.1153846153846, 91.56793206793206), (102.99650349650351, 78.46103896103895), (102.51698301698302, 64.23526473526474), (101.3981018981019, 49.84965034965035), (99.16033966033966, 32.586913086913086), (117.38211788211787, 91.4080919080919), (117.54195804195804, 77.02247752247752), (116.1033966033966, 61.51798201798201), (115.3041958041958, 45.693806193806196), (113.545954045

In [260]:
# Compute local max in neighborhood
def pick_dot(img, click_point_xy, search_radius=10):
	h, w = img.shape
	clk_x, clk_y = click_point_xy

	# Take slice of image
	img_slice = img[
		max(0, clk_y-search_radius+1) : min(h, clk_y+search_radius),
		max(0, clk_x-search_radius+1) : min(w, clk_x+search_radius),
	]

	# Insert slice into image
	sub_img = np.zeros((2*search_radius-1, 2*search_radius-1), dtype=img.dtype)
	
	min_y = clk_y-search_radius-1
	min_x = clk_x-search_radius-1

	ins_x = 0 if min_x >= 0 else -1*min_x-1
	ins_y = 0 if min_y >= 0 else -1*min_y-1
	sub_img[ins_y:, ins_x:] = img_slice

	# Circular mask
	mask = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*search_radius-1, 2*search_radius-1))
	sub_img = cv2.bitwise_and(sub_img, sub_img, mask=mask)

	# Compute max and find centroid in a tie
	max_val = max(sub_img.flatten())
	peak_y, peak_x = np.mean(np.where(sub_img == max_val), 1).astype("float32")
	return clk_x+(peak_x-search_radius+1), clk_y+(peak_y-search_radius+1)

# Get precise points
for k in points:
	path = os.path.join(folder, k)
	img = cv2.imread(path,  cv2.IMREAD_GRAYSCALE)

	corrected = []
	for p in points[k]:
		px, py = np.round(p)
		corrected.append(pick_dot(img, (int(px), int(py)), 8))
		print((px, py), "-->", corrected[-1])
	else:
		points[k] = corrected

	

(66.0, 90.0) --> (66.0, 90.0)
(65.0, 78.0) --> (64.0, 78.0)
(63.0, 65.0) --> (62.0, 65.0)
(61.0, 52.0) --> (61.0, 52.0)
(60.0, 39.0) --> (60.0, 39.0)
(77.0, 91.0) --> (77.0, 91.0)
(76.0, 79.0) --> (76.0, 79.0)
(75.0, 64.0) --> (74.0, 65.0)
(73.0, 51.0) --> (73.0, 51.0)
(72.0, 38.0) --> (71.0, 37.0)
(90.0, 91.0) --> (90.0, 91.0)
(89.0, 77.0) --> (89.0, 78.0)
(88.0, 65.0) --> (87.5, 64.0)
(87.0, 50.0) --> (86.0, 49.0)
(84.0, 34.0) --> (84.0, 35.0)
(104.0, 92.0) --> (104.0, 91.0)
(103.0, 78.0) --> (103.0, 78.0)
(103.0, 64.0) --> (102.0, 63.0)
(101.0, 50.0) --> (100.0, 47.66666650772095)
(99.0, 33.0) --> (98.0, 33.0)
(117.0, 91.0) --> (118.0, 91.0)
(118.0, 77.0) --> (118.0, 76.0)
(116.0, 62.0) --> (117.0, 62.0)
(115.0, 46.0) --> (115.33333349227905, 46.33333349227905)
(114.0, 31.0) --> (113.0, 31.0)
(131.0, 89.0) --> (131.0, 90.0)
(133.0, 75.0) --> (132.0, 76.0)
(131.0, 60.0) --> (131.0, 60.0)
(130.0, 44.0) --> (130.0, 45.0)
(128.0, 28.0) --> (128.0, 30.0)
(144.0, 87.0) --> (144.0, 88.0)
(

In [261]:
# Save points as CSV
folder = "./thermal_dots/"

for k in points:
	filename = k.split(".")[0] + ".csv"
	file_obj = open(os.path.join(folder, filename), 'w', newline='', encoding='utf-8')
	writer = csv.writer(file_obj)
	
	for p in points[k]:
		writer.writerow(p)
	
	file_obj.close()

In [265]:
# Draw Points
# image_folder = "../calibration_data/thermal/"
# point_folder = "./thermal/"

image_folder = "../calibration_data/thermal_dots_out1/"
point_folder = "./thermal_dots/"

cv2.namedWindow("img", cv2.WINDOW_NORMAL)

images = os.listdir(image_folder)
for f in images:
	name = f.split(".")[0]
	points = np.loadtxt(os.path.join(point_folder, f"{name}.csv"), delimiter=",", dtype=float)
	points = np.array([np.array([r]) for r in points]).astype('float32')

	img = cv2.imread(os.path.join(image_folder, f), cv2.IMREAD_UNCHANGED)
	out = cv2.drawChessboardCorners(cv2.cvtColor(img, cv2.COLOR_GRAY2BGR), (5, 7), points, True)
	
	cv2.imshow("img", out)
	while cv2.waitKey(0) != ord('q'): continue

cv2.destroyAllWindows()

In [273]:
# Create psuedo-image
image_folder = "../calibration_data/thermal_dots/"
point_folder = "./thermal_dots/"

cv2.namedWindow("img", cv2.WINDOW_NORMAL)
cv2.namedWindow("out", cv2.WINDOW_NORMAL)

images = os.listdir(image_folder)
for f in images:
	name = f.split(".")[0]
	points = np.loadtxt(os.path.join(point_folder, f"{name}.csv"), delimiter=",", dtype=float)
	points = np.array([(int(round(x)), int(round(y))) for x, y in points])

	img = cv2.imread(os.path.join(image_folder, f), cv2.IMREAD_UNCHANGED)
	out = 255*np.ones_like(img)

	for p in points:
		print(p)
		out = cv2.circle(out, p, 2, (0,0,0), -1)
	
	cv2.imwrite(os.path.join("../calibration_data/thermal_dots_binary/", f), out)
	
	cv2.imshow("img", img)
	cv2.imshow("out", out)
	while cv2.waitKey(0) != ord('q'): continue

cv2.destroyAllWindows()

[66 90]
[64 78]
[62 65]
[61 52]
[60 39]
[77 91]
[76 79]
[74 65]
[73 51]
[71 37]
[90 91]
[89 78]
[88 64]
[86 49]
[84 35]
[104  91]
[103  78]
[102  63]
[100  48]
[98 33]
[118  91]
[118  76]
[117  62]
[115  46]
[113  31]
[131  90]
[132  76]
[131  60]
[130  45]
[128  30]
[144  88]
[145  74]
[145  59]
[143  43]
[141  29]
[20 83]
[17 71]
[15 58]
[13 44]
[13 30]
[31 83]
[28 69]
[26 56]
[25 42]
[24 28]
[43 82]
[41 68]
[39 54]
[38 39]
[37 26]
[57 80]
[55 67]
[53 52]
[52 38]
[52 24]
[71 79]
[69 65]
[68 51]
[67 36]
[66 22]
[84 77]
[83 64]
[82 50]
[81 36]
[80 21]
[97 75]
[97 62]
[96 48]
[95 35]
[94 21]
[ 60 100]
[60 89]
[61 78]
[62 66]
[62 55]
[ 71 102]
[71 90]
[71 78]
[72 66]
[72 54]
[ 82 103]
[83 91]
[83 78]
[83 66]
[84 54]
[ 95 104]
[95 91]
[96 78]
[96 65]
[96 53]
[108 104]
[109  91]
[109  78]
[109  64]
[108  52]
[121 104]
[122  91]
[122  77]
[122  64]
[121  51]
[134 103]
[135  90]
[135  76]
[134  63]
[133  50]
[63 93]
[61 81]
[59 67]
[57 52]
[56 38]
[76 93]
[74 80]
[73 65]
[71 50]
[69 35]
[90 